In [1]:
#!pip3 install julia
#import julia
#julia.install()

#Temporal fix because this python installation does not have libpython
# from julia.api import Julia
# jl = Julia(compiled_modules=False)
#!pip3 install numpy
# !pip3 install matplotlib

import matplotlib.pyplot as plt
from npodjlEnv import NpodJLEnv
import numpy as np
env = NpodJLEnv()
env.ver()

0.6

In [2]:
def argmaxQ(Q, s, n_actions):
    if s in Q.keys():
        print("Policy: Greedy")
        return(np.argmax(Q[s]))
    else:
        print("Policy: Random")
        return(np.random.randint(0, n_actions))

In [3]:
def policy_greedy(Q, s, epsilon, n_actions):
    if np.random.rand() < epsilon:
        return(argmaxQ(Q,s,n_actions))
    else:
        print("Policy: Random")
        return(np.random.randint(0, n_actions))

In [4]:
def SARSA(env, alpha, gamma, epsilon, Q, max_cylces):
    n_actions = len(env.actions())
    r = [None] * (max_cycles+1)
    s = [None] * (2*(max_cycles+1))
    a = [None] * (2*max_cycles)
    
    t=0
    s[t] = env.encoded_state()
    a[t] = policy_greedy(Q,s[t], epsilon, n_actions)
    obs = env.run(a[t])
    r[t] = obs['reward']
    s[t+1] = env.encoded_state() 
    if not s[t] in Q.keys():
        Q[s[t]] = [0]*n_actions
    try:
        while t < max_cycles:
            print("State: ",env.encoded_state())
            a[t+1] = policy_greedy(Q,s[t+1], epsilon, n_actions)
            print("Action: ", a[t])
            obs = env.run(a[t])
            print("Reward: ", obs['reward'])
            print("-----")
            r[t+1] = obs['reward']
            s[t+2] = env.encoded_state()
            if not s[t+1] in Q.keys():
                Q[s[t+1]] = [0]*n_actions
            Q[s[t]][a[t]] = Q[s[t]][a[t]] + alpha * ( r[t] + (gamma * Q[s[t+1]][a[t+1]] ) - Q[s[t]][a[t]] )
            t += 1
        print(r)
        print("Total Reward: ", sum(r))
        return(sum(r))
            
    except KeyboardInterrupt:
        print("Press Ctrl-C to terminate while statement")
        pass
    return(s,a,r,Q)

In [5]:
Q = dict()

In [6]:
env.reset()
alpha = 1
gamma = 0.99
epsilon = 0.5
max_cycles = 20
#sol = SARSA(env, alpha, gamma, epsilon, Q, max_cycles)

In [7]:
Q

{}

In [ ]:
total_reward=[]
for i in range(1,40):
        print(i)
        env.reset()
        total_reward.append(SARSA(env, 0.9, 1, 1 - 1/i, Q, 20))

In [ ]:
plt.plot(total_reward)